In this notebook, we will try to tune the OPTICS clustering algorithm. 

Spoilers: This notebook is kind of messy because I started modifying the hyperparameter without much technique. The verdict is that it is difficult to find optimal hyperparameters because **they vary by region (e.g. if users take trips in sprawling suburbs or densely packed cities)**. 

### imports

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from uuid import UUID

# hack because jupyter notebook doesn't work properly through my vscode for
# some reason and therefore cant import stuff from emission? remove this before
# pushing
###
import sys

sys.path.append('/Users/hlu2/Documents/GitHub/e-mission-server/')
###

import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq
import emission.core.get_database as edb
import mapping


### load data

In [ ]:
# to see the same outputs I described, put in the unique tokens for these users
email0 = "replace this"  # shankari
email1 = "replace this"  # tom
user0 = list(edb.get_uuid_db().find({"user_email": email0}))[0]['uuid']
user1 = list(edb.get_uuid_db().find({"user_email": email1}))[0]['uuid']
user2 = UUID('replace this')  # hannah

In [ ]:
all_users = esta.TimeSeries.get_uuid_list()
user_list = np.append([user0, user1, user2],
                      np.random.choice(all_users, size=10, replace=False))
confirmed_trip_df_map = {}
labeled_trip_df_map = {}
expanded_labeled_trip_df_map = {}
expanded_all_trip_df_map = {}
for i in range(len(user_list)):
    u = user_list[i]
    print(u)
    ts = esta.TimeSeries.get_time_series(u)
    ct_df = ts.get_data_df("analysis/confirmed_trip")

    confirmed_trip_df_map[i] = ct_df
    labeled_trip_df_map[i] = esdtq.filter_labeled_trips(ct_df)
    expanded_labeled_trip_df_map[i] = esdtq.expand_userinputs(
        labeled_trip_df_map[i])
    expanded_all_trip_df_map[i] = esdtq.expand_userinputs(
        confirmed_trip_df_map[i])


In [ ]:
n_trips_df = pd.DataFrame([[
    user_list[i],
    len(confirmed_trip_df_map[i]),
    len(labeled_trip_df_map[i])
] for i in range(len(user_list))],
                          columns=["user_id", "all_trips", "labeled_trips"])
n_trips_df

### tuning

#### initial tuning of radii and minpts (no xi modification)

optics with min samples of 2

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=2)
fig

optics with min samples of 3

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=5)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=10)
fig

In [ ]:
# importlib.reload(mapping)
print(len(expanded_labeled_trip_df_map[0]))
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=int(
                                0.1 * len(expanded_labeled_trip_df_map[0])))
fig

In [ ]:
# importlib.reload(mapping)
print(len(expanded_labeled_trip_df_map[0]))
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=int(
                                0.07 * len(expanded_labeled_trip_df_map[0])))
fig

In [ ]:
importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=20)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(
    expanded_labeled_trip_df_map[0],
    alg='OPTICS',
    loc_type='end',
    radii=[50, 100, 150, 200, 250, 300, 400, 500],
    optics_min_samples=40,
)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(
    expanded_labeled_trip_df_map[0],
    alg='OPTICS',
    loc_type='end',
    radii=[50, 100, 150, 200, 250, 300, 400, 500],
    optics_min_samples=30,
)
fig

#### tuning xi

Ok, here's the problem: the super dense 'home' cluster isn't being recognized as such, whether I vary radius or minPts. It's forming really tiny clusters and not expanding beyond those. If I raise minPts too high, then we won't see other smaller clusters. However, increasing the radius doesn't seem to help much. 

minPts = 2 is much too small as it yields clusters that mostly only have 2-3 trips. 
minPts = 5 is able to nicely distinguish library and shopping but doesn't recognize the massive home cluster. (It found a sparse cluster of noise off the primary home cluster though.)
minPts = 10 still doesn't recognize the home cluster, but is making a concerningly large shopping cluster at a low radius without recognizing the library cluster as easily. 

So I think what we should try is to lower the steepness threshold for points to be considered part of a cluster. My hypothesis is that the steepness increases too fast around the points in the home cluster and therefore they look like a bunch of tiny separate clusters. 

As a side note, I'm not really sure what's the best way to tune this algorithm right now. Immediately I think of cross-validation, but I wonder what score we should use. Harmonic mean of (modified) homogeneity with cluster %? 

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=3,
                            optics_cluster_method='dbscan')
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=3,
                            optics_xi=0.05)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=3,
                            optics_xi=0.2)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=3,
                            optics_xi=0.4)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=3,
                            optics_xi=0.6)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=3,
                            optics_xi=0.7)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=3,
                            optics_xi=0.9)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[1],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=3,
                            optics_xi=0.5)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[1],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=2,
                            optics_xi=0.7)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[1],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=3,
                            optics_xi=0.7)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[1],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300, 400, 500],
                            optics_min_samples=4,
                            optics_xi=0.6)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[1],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=3,
                            optics_xi=0.85)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[1],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=3,
                            optics_xi=0.9)
fig

For these past few maps with xi <=0.8, what I don't understand is why the shopping cluster keeps disappearing at/above 100m even though the cluster is found at 50m...
Ok, looks like xi=0.9 yields better and more intuitive results - as the radius increases, nearby clusters get merged into each other. Since xi is the minimum steepness at which a point is cut off from a cluster, xi essentially seems to represent the algorithm's sensitivity to density variation; a xi value closer to 1 would make the algorithm more similar to DBSCAN, which is entirely insensitive to density change. 

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[1],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=2,
                            optics_xi=0.9)
fig

radius=100m, minPts=2, xi=0.9 looks really good for this user. When the minpts was 3, we missed some two-trips clusters that were definitely at the same location and which would have provided useful information. 

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=2,
                            optics_xi=0.9)
fig

unfortunately, those parameters failed to find the super dense home cluster :( also, the clusters around library and shopping disappeared as the radius increased ??

increasing minPts to 3 seems to fix things:

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=3,
                            optics_xi=0.9)
fig

#### tuning xi for other users

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[2],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=2,
                            optics_xi=0.9)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[2],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=3,
                            optics_xi=0.9)
fig

Similar results: minpts=2 failed to find the important, dense clusters. Increasing to minpts=3 helped. Unfortunately, it was only able to distinguish between school/meal at 50m; by 100m, they were all clumped together. SO OPTICS helps with a little bit of density variation compared to DBSCAN, but not really for dramatic variation in density. We could try lowering xi a little to see if that helps. 

(Hmm, lowering xi below 0.85 for user[1] and user[3] would make some clusters disappear.) So xi=0.85 is not enough to distinguish between school/meal but xi=0.8 fails to find any proper school/meal clusters at all. 

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[3],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=3,
                            optics_xi=0.85)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[4],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=3,
                            optics_xi=0.85)
fig

At minpts=2, rad=100, xi=0.9, this user had a dense home cluster that wasnt picked up. It also grouped a lot of noisy points into a work cluster located in a downtown area. 

Bumping minpts up to 3 helped us find the big home cluster. The work cluster was still pretty noisy. Let's try dropping xi down to 0.85. 

Ok, so dropping xi down to 0.85 kept clusters pretty similar. Home is still well-identified, work is still noisy. 

User[5] only had 9 labeled points. We found 2 two-trip clusters when minPts=2, and nothing when minPts=3. 

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[5],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=3,
                            optics_xi=0.85)
fig

Users [6-9] had no labeled data. I think we found Tom by accident again as user 10. User 11 had a single trip.

User 13 has 28 data points, and it was pretty spread out around the city, not really any repeated trips. We found 2 home clusters nicely though. 

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[13],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=2,
                            optics_xi=0.9)
fig

In [ ]:
# importlib.reload(mapping)
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[12],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            optics_min_samples=3,
                            optics_xi=0.85)
fig

Verdict: based purely on anecdotal evidence, it looks like OPTICS produces okay-ish results with parameters of minPts=3, xi around 0.85-0.9, radius around 100m. We should do some parameter tuning on a mass scale to see how it performs for a wide variety of users. 

Still, we're missing useful 2-trip clusters. Also, we don't know how the clusters look when we add unlabeled points. And I think it makes sense to use a semi-supervised alg to leverage any label information if we have it. 

### Check clustering with unlabeled trips

In [ ]:
fig = mapping.plot_clusters(expanded_all_trip_df_map[1],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            cluster_unlabeled=True,
                            plot_unlabeled=True,
                            optics_min_samples=3,
                            optics_xi=0.85)
fig

In [ ]:
fig = mapping.plot_clusters(expanded_all_trip_df_map[3],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            cluster_unlabeled=True,
                            plot_unlabeled=True,
                            optics_min_samples=3,
                            optics_xi=0.9)
fig

In [ ]:
fig = mapping.plot_clusters(expanded_all_trip_df_map[3],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            cluster_unlabeled=True,
                            plot_unlabeled=True,
                            optics_min_samples=3,
                            optics_xi=0.9)
fig

In [ ]:
fig = mapping.plot_clusters(expanded_all_trip_df_map[4],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            cluster_unlabeled=True,
                            plot_unlabeled=True,
                            optics_min_samples=3,
                            optics_xi=0.85)
fig

In [ ]:
fig = mapping.plot_clusters(expanded_all_trip_df_map[5],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            cluster_unlabeled=True,
                            plot_unlabeled=True,
                            optics_min_samples=3,
                            optics_xi=0.85)
fig

Interesting. for user 6, I think the introduction of noisy unlabeled points caused it to merge home/library/park. Yep, if we don't use unlabeled points to determine cluster bounds then it helps.

In [ ]:
fig = mapping.plot_clusters(expanded_all_trip_df_map[6],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            cluster_unlabeled=True,
                            plot_unlabeled=True,
                            optics_min_samples=3,
                            optics_xi=0.8)
fig

In [ ]:
fig = mapping.plot_clusters(expanded_all_trip_df_map[6],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            cluster_unlabeled=False,
                            plot_unlabeled=True,
                            optics_min_samples=3,
                            optics_xi=0.8)
fig

at xi=0.85, we failed to find a big home cluster for user7. Found it with x=0.9. 

In [ ]:
fig = mapping.plot_clusters(expanded_all_trip_df_map[7],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            cluster_unlabeled=True,
                            plot_unlabeled=True,
                            optics_min_samples=3,
                            optics_xi=0.9)
fig

In [ ]:
fig = mapping.plot_clusters(expanded_all_trip_df_map[8],
                            alg='OPTICS',
                            loc_type='end',
                            radii=[50, 100, 150, 200, 250, 300],
                            cluster_unlabeled=True,
                            plot_unlabeled=True,
                            optics_min_samples=3,
                            optics_xi=0.85)
fig